In [117]:
import sys  
sys.path.insert(0, '../../utilities/')

import utils
import time

In [66]:
import string

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [53]:
#constants
b="b"
ZERO="0"
ONE="1"
L="L"
R="R"
ENDSTATE="e"

In [5]:
def addBlanks(tape:string, numberOfBlanks:int=10):
    if numberOfBlanks <= 0:
        print("\nNumber of blanks cannot be less than zero")
        return

    if tape is None:
        tape=''

    for blank in range(numberOfBlanks-1):
        tape=BLANK+tape+BLANK
    return tape


In [82]:
class StateAction:
    def __init__(self, currentSymbol, newSymbol,stepDirection, newState,name="action"):
        self.name = name
        self.currentSymbol=currentSymbol
        self.newSymbol = newSymbol
        self.stepDirection = stepDirection
        self.newState = newState

    def getActionString(self):
        return bcolors.HEADER+self.currentSymbol+self.newSymbol+bcolors.ENDC+bcolors.WARNING+self.stepDirection+bcolors.ENDC+bcolors.OKBLUE+self.newState+bcolors.ENDC

    def printAction(self):
        print(self.getActionString())

    def __str__(self):
        return self.name


        
class EndAction(StateAction):
     def __init__(self):
        super().__init__("",ENDSTATE, "","",ENDSTATE)

StateAction(ONE,ZERO,L,"q1").printAction()
EndAction().printAction()

10Lq1
e


In [99]:
class State:
    def __init__(self, name, oneAction,zeroAction,blankAction,displayName="What does this state do?"):
        self.name = name
        self.oneAction = oneAction
        self.zeroAction = zeroAction
        self.blankAction = blankAction
        self.displayName = displayName
        self.actions =[oneAction,zeroAction,blankAction]

    def findAction(self,symbol):
        for action in self.actions:
            if action.currentSymbol== symbol:return action

    def printActions(self):
        for action  in self.actions:
            action.printAction()

    def printState(self):
        print(self.name+":"+"\t"+self.displayName)
        for action  in self.actions:
            print("\t"+action.getActionString())


    def printInstruction(self, symbol):
        action =self.findAction(symbol)
        print(bcolors.OKBLUE+self.name+bcolors.ENDC+action.getActionString())

    def __str__(self):
        return self.name

class EndState(State):
    def __init__(self):
        self.name=ENDSTATE
        self.displayName="STOP"
        self.actions=[EndAction()]
    def findAction(self,symbol):
            return self.actions[0]
    def printActions(self):
            print(ENDSTATE)   
    def printState(self):
            print(ENDSTATE)   

In [97]:

state1 = State("q1",StateAction(ONE,ZERO,L,"q1"),StateAction(ZERO,ZERO,R,ENDSTATE),StateAction(b,b,L,ENDSTATE), "Check if there are one more 1 to copy")
state2 = State("q2",StateAction(ONE,ZERO,L,"q1"),StateAction(ZERO,ZERO,R,ENDSTATE),StateAction(b,b,L,ENDSTATE),"Copy a one at the left end of the output")
state3 = State("q3",StateAction(ONE,ZERO,L,"q1"),StateAction(ZERO,ZERO,R,ENDSTATE),StateAction(b,b,L,ENDSTATE),"Search for the first 0 to the right")
#state.printActions()
#state.printInstruction(ONE)
#state.printInstruction(ZERO)
#state.printInstruction(b)
state1.printState()

states = [state1,state2,state3]

q1:	Check if there are one more 1 to copy
	10Lq1
	00Re
	bbLe


In [133]:
from re import S


class TuringMachine:




   def __init__(self,states,startingState, startingPosition=0):
        self.states =states
        self.states.append(EndState())
        self.startingState =startingState
        self.startingPosition =startingPosition
        self.init()

   def init(self):
      self.input=""
      self.changedCells=""
      self.currentState=""
      self.currentPosition=0
      self.currentSymbol=""
      self.tape=""
      self.count=0
      self.currentStateObject=None


   
   def initNewRun(self, input):
        self.init()
        self.input=input
        self.changedCells=input
        self.currentState=self.startingState
        self.currentPosition=self.startingPosition
        self.currentSymbol=""
        self.tape =input
        self.count=0
        self.currentStateObject=None
        self.addBlanks()

   def printStates(self):
      for state in self.states:
         state.printState()

   def addBlanks(self, numberOfBlanks:int=4):
      if numberOfBlanks <= 0:
         print("\nNumber of blanks cannot be less than zero")
         return

      if self.tape is None:
         self.tape=''

      for blank in range(numberOfBlanks):
         self.tape=b+self.tape+b

      self.currentPosition = self.currentPosition+numberOfBlanks
      
      return self.tape

   def printDump(self):
      print(self.input)
      print(self.changedCells)
      print(self.currentState)
      print(self.currentPosition)
      print(self.currentSymbol)
      print(self.tape)

   def readCurrentSymbol(self):
      self.currentSymbol=self.tape[self.currentPosition]

   def setCurrentStateObject(self):
         for s in self.states:
            if s.name ==self.currentState: 
               self.currentStateObject=s

   def getActionFromCurrentState(self):
      action=self.currentStateObject.findAction(self.currentSymbol)
      return action

   def printNewSymbol(self,newSymbol):
      self.tape = self.tape[:self.currentPosition] + newSymbol + self.tape[self.currentPosition+1:]

   def executeCurrentState(self):
      self.readCurrentSymbol()
      self.setCurrentStateObject()
      action =self.getActionFromCurrentState()
      if self.currentSymbol == action.currentSymbol:
         self.printNewSymbol(action.newSymbol)
         self.currentState = action.newState
         if(action.stepDirection==L):
            self.currentPosition=self.currentPosition-1
         else:
            self.currentPosition=self.currentPosition+1

         

   def step(self):
      print("step "+str(self.count)+"\t"+self.tape)
      self.executeCurrentState()


   def run(self, input):
      self.initNewRun(input)
      while True:
         self.step()
         #self.printDump()
         self.getActionFromCurrentState()
         self.count=self.count+1
         time.sleep(5)



cu=TuringMachine(states,"q1")
#cu.printStates()
cu.run("11")
cu.printDump()









bbbb11bbbb
step 0
1
bbbb01bbbb
step 1
0
bbbb01bbbb
step 2
0
bbbb01bbbb
step 3
0
bbbb01bbbb
step 4
0
bbbb01bbbb
step 5
0
bbbb01bbbb
step 6
0
bbbb01bbbb
step 7
0
bbbb01bbbb
step 8
0
bbbb01bbbb
step 9
0
bbbb01bbbb
step 10
0
bbbb01bbbb
step 11
0
bbbb01bbbb
step 12
0
bbbb01bbbb
step 13
0
bbbb01bbbb
step 14
0
bbbb01bbbb
step 15
0
bbbb01bbbb
step 16
0
bbbb01bbbb
step 17
0
bbbb01bbbb
step 18
0
bbbb01bbbb
step 19
0
bbbb01bbbb
step 20
0
bbbb01bbbb
step 21
0
bbbb01bbbb
step 22
0
bbbb01bbbb
step 23
0
bbbb01bbbb
step 24
0
bbbb01bbbb
step 25
0
bbbb01bbbb
step 26
0
bbbb01bbbb
step 27
0
bbbb01bbbb
step 28
0
bbbb01bbbb
step 29
0
bbbb01bbbb
step 30
0
bbbb01bbbb
step 31
0
bbbb01bbbb
step 32
0
bbbb01bbbb
step 33
0
bbbb01bbbb
step 34
0
bbbb01bbbb
step 35
0
bbbb01bbbb
step 36
0
bbbb01bbbb
step 37
0
bbbb01bbbb
step 38
0
bbbb01bbbb
step 39
0
bbbb01bbbb
step 40
0
bbbb01bbbb
step 41
0
bbbb01bbbb
step 42
0
bbbb01bbbb
step 43
0
bbbb01bbbb
step 44
0
bbbb01bbbb
step 45
0
bbbb01bbbb
step 46
0
bbbb01bbbb
step 47
0
bb

KeyboardInterrupt: 

In [ ]:
class Tape:

   def __init__(self,input):
        self.input =input
        self.changedCells=input
        self.tape =self.addBlanks()

   def addBlanks(tape:string, numberOfBlanks:int=10):
      if numberOfBlanks <= 0:
         print("\nNumber of blanks cannot be less than zero")
         return

      if tape is None:
         tape=''

      for blank in range(numberOfBlanks-1):
         tape=b+tape+b
      return tape





In [26]:
input = "0101010101"
input=addBlanks(input)
input=addBlanks(input)
input=addBlanks(input)
input=addBlanks(input)
addBlanks(input)

'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb0101010101bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb'

In [104]:
text = 'abcdefg'
text = text[:1] + 'Z' + text[2:]
print(text)

aZcdefg
